# Distributional Counterfactual Explanation

In [41]:
import pandas as pd
import numpy as np
import seaborn as sns

import torch.optim as optim
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder

from models.mlp import BlackBoxModel

pd.set_option('display.max_columns', None)

%reload_ext autoreload
%autoreload 2

## Read and Process Data

In [42]:
df_ = pd.read_csv('data/hotel_booking/hotel_bookings.csv')
df = df_.copy()
target_name = 'is_canceled'
target = df[target_name]

In [45]:
# Initialize a label encoder
label_encoder = LabelEncoder()
label_mappings = {}


# Convert categorical columns to numerical representations using label encoding
for column in df.columns:
    if df[column].dtype == 'object':
        # Handle missing values by filling with a placeholder and then encoding
        df[column] = df[column].fillna('Unknown')
        df[column] = label_encoder.fit_transform(df[column])
        label_mappings[column] = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))



# For columns with NaN values that are numerical, we will impute them with the median of the column
for column in df.columns:
    if df[column].isna().any():
        median_val = df[column].median()
        df[column].fillna(median_val, inplace=True)

# Display the first few rows of the transformed dataframe
df.head()


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,1,0,342,2015,5,27,1,0,0,2,0.0,0,0,135,3,1,0,0,0,2,2,3,0,14.0,179.0,0,2,0.0,0,0,1,15
1,1,0,737,2015,5,27,1,0,0,2,0.0,0,0,135,3,1,0,0,0,2,2,4,0,14.0,179.0,0,2,0.0,0,0,1,15
2,1,0,7,2015,5,27,1,0,1,1,0.0,0,0,59,3,1,0,0,0,0,2,0,0,14.0,179.0,0,2,75.0,0,0,1,46
3,1,0,13,2015,5,27,1,0,1,1,0.0,0,0,59,2,0,0,0,0,0,0,0,0,304.0,179.0,0,2,75.0,0,0,1,46
4,1,0,14,2015,5,27,1,0,2,2,0.0,0,0,59,6,3,0,0,0,0,0,0,0,240.0,179.0,0,2,98.0,0,1,1,76


## Model Training

In [4]:
features = [
    'hotel', 
    'lead_time', 
    'arrival_date_year', 
    'arrival_date_month',
    'arrival_date_week_number', 
    'arrival_date_day_of_month',
    'stays_in_weekend_nights', 
    'stays_in_week_nights', 
    'adults', 
    'children',
    'babies', 
    'meal', 
    'country', 
    'market_segment', 
    'distribution_channel',
    'is_repeated_guest', 
    'previous_cancellations',
    'previous_bookings_not_canceled', 
    'reserved_room_type',
    'assigned_room_type', 
    'booking_changes', 
    'deposit_type', 
    'agent',
    'company', 
    'days_in_waiting_list', 
    'customer_type', 
    'adr',
    'required_car_parking_spaces', 
    'total_of_special_requests'
]

df_X = df[features].copy()
df_y = target

In [5]:
seed = 42

np.random.seed(seed)  # for reproducibility


# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=seed)

std = X_train.std()
mean = X_train.mean()

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# X_train, X_test, y_train, y_test = X_train.values, X_test.values, y_train.values, y_test.values

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.FloatTensor(y_train.values).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test.values)
y_test_tensor = torch.FloatTensor(y_test.values).view(-1, 1)

# Initialize the model, loss function, and optimizer
model = BlackBoxModel(input_dim=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Evaluate on test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)

    # Convert outputs to binary using 0.5 as threshold
    y_pred_tensor = (test_outputs > 0.5).float()
    correct_predictions = (y_pred_tensor == y_test_tensor).float().sum()
    accuracy = correct_predictions / y_test_tensor.shape[0]

accuracy.item()

0.8287126421928406

## Counterfactual Explanation

In [6]:
sample_num = 100
delta = 0.1
alpha = 0.05
N=10
explain_columns = [
    'lead_time', 
    'booking_changes', 
    'total_of_special_requests',
    'is_repeated_guest',
]

indice = (X_test.sample(sample_num)).index

df_explain = X_test.loc[indice]

# X = X_test.loc[indice].values
y = model(torch.FloatTensor(df_explain.values))

y_target = torch.distributions.beta.Beta(0.1, 0.9).sample((sample_num,))

y_true = y_test.loc[indice]

In [7]:
from explainers.dce import DistributionalCounterfactualExplainer

explainer = DistributionalCounterfactualExplainer(
    model=model, 
    df_X=df_explain, 
    explain_columns=explain_columns,
    y_target=y_target, 
    lr=1e-1, 
    n_proj=N,
    delta=delta)

DEBUG:root:test


In [8]:
np.sqrt(explainer.wd.distance(y, y_target, delta=delta)[0].item())

0.4648085508723789

In [9]:
explainer.wd.distance_interval(y, y_target, delta=delta, alpha=0.10)

(0.3268023445169612, 0.5483401304466742)

In [10]:
explainer.optimize(U_1=0.5, U_2=0.5, l=0.2, r=1, max_iter=20, tau=1e3)

INFO:root:Optimization started
c:\Users\leiyo\GitHub\distributional-counterfactual-explanation\explainers\distances.py:303: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
c:\Users\leiyo\GitHub\distributional-counterfactual-explanation\explainers\distances.py:304: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)
c:\Users\leiyo\GitHub\distributional-counterfactual-explanation\explainers\distances.py:329: RuntimeWarning: invalid value encountered in power
  return np.power(SW_lower, 1 / r), np.power(SW_upper, 1 / r)
INFO:root:U_1-Qu_upper=nan, U_2-Qv_upper=-0.06745401565979114
INFO:root:eta=1, l=0.2, r=1
INFO:root

In [18]:
X_s = explainer.best_X[:, explainer.explain_indices].clone()
X_t = explainer.X_prime[:, explainer.explain_indices].clone()

In [19]:
np.sqrt(explainer.swd.distance(X_s, X_t, delta)[0].item())

0.4535171357621716

In [20]:
explainer.swd.distance_interval(X_s, X_t, delta=delta, alpha=alpha)

c:\Users\leiyo\GitHub\distributional-counterfactual-explanation\explainers\distances.py:303: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
c:\Users\leiyo\GitHub\distributional-counterfactual-explanation\explainers\distances.py:304: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)


(0.33485993177552675, 0.48679789065060847)

In [46]:
factual_X = df[df_X.columns].loc[indice].copy()
counterfactual_X = pd.DataFrame(explainer.best_X.detach().numpy() * std[df_X.columns].values + mean[df_X.columns].values, columns=df_X.columns)

dtype_dict = df.dtypes.apply(lambda x: x.name).to_dict()
for k, v in dtype_dict.items():
    if k in counterfactual_X.columns:
        if v[:3] == 'int':
            counterfactual_X[k] = counterfactual_X[k].round().astype(v)
        else:
            counterfactual_X[k] = counterfactual_X[k].astype(v)


factual_y = pd.DataFrame(y.detach().numpy(),columns=[target_name], index=factual_X.index)
counterfactual_y = pd.DataFrame(explainer.y.detach().numpy(),columns=[target_name], index=factual_X.index)

In [49]:
# Now, reverse the label encoding using the label_mappings
for dft in [factual_X, counterfactual_X]:
    for column, mapping in label_mappings.items():
        if column in dft.columns:
            # Invert the label mapping dictionary
            inv_mapping = {v: k for k, v in mapping.items()}
            # Map the encoded labels back to the original strings
            dft[column] = dft[column].map(inv_mapping)


In [50]:
counterfactual_X

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,NaN,80,2017,May,21,22,1,1,2,-3.946223e-09,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,No Deposit,314.999989,179.000000,0,Transient-Party,110.000000,0,1
1,NaN,130,2016,March,12,15,0,2,1,-3.946223e-09,0,BB,PRT,Corporate,Corporate,0,0,0,A,D,0,No Deposit,14.000001,113.000003,0,Transient,29.999999,0,0
2,NaN,123,2017,July,27,8,1,1,2,-3.946223e-09,0,BB,ESP,Direct,Direct,0,0,0,A,A,0,No Deposit,14.000001,179.000000,0,Transient,108.000000,0,1
3,NaN,63,2016,September,38,15,0,3,2,-3.946223e-09,0,BB,GBR,Groups,Direct,0,0,0,A,E,0,No Deposit,14.000001,223.000000,0,Transient-Party,68.000001,0,0
4,NaN,138,2015,August,34,17,1,1,2,-3.946223e-09,0,BB,PRT,Groups,TA/TO,0,1,0,A,A,0,Non Refund,1.000003,179.000000,0,Transient-Party,62.000001,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,110,2015,August,34,20,2,8,2,-3.946223e-09,0,BB,ESP,Offline TA/TO,TA/TO,0,0,0,A,A,1,No Deposit,242.999997,179.000000,0,Transient,104.000000,0,1
96,NaN,143,2015,September,36,4,1,2,2,-3.946223e-09,0,BB,PRT,Offline TA/TO,TA/TO,0,1,0,A,A,0,No Deposit,2.999998,179.000000,0,Transient-Party,80.000000,0,1
97,NaN,66,2015,December,52,24,0,1,1,-3.946223e-09,0,BB,MUS,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.000002,179.000000,0,Transient,80.999999,0,1
98,NaN,96,2017,May,20,14,2,4,2,-3.946223e-09,0,SC,IRL,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.000002,179.000000,0,Transient-Party,143.999999,0,1


In [ ]:
pd.DataFrame({
    'factual_y': factual_y[target_name].values,
    'counterfactual_y': counterfactual_y[target_name].values,
})

In [ ]:
counterfactual_X.index = factual_X.index
counterfactual_X[target_name] = counterfactual_y

In [ ]:
factual_X[target_name] = factual_y

In [ ]:
factual_X.head(5)

In [ ]:
counterfactual_X.head(5)

In [ ]:
check_column = 'lead_time'
pd.DataFrame({
    'factual': factual_X[check_column].values, 
    'counterfactual': counterfactual_X[check_column].values
    })

In [ ]:
factual_X[check_column].mean(), counterfactual_X[check_column].mean()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Sample matrix for demonstration
matrix = explainer.wd.nu.numpy()

plt.figure(figsize=(10, 8))
plt.imshow(matrix, cmap='viridis')
plt.colorbar()
plt.title("Heatmap of the Matrix")
plt.show()
